In [ ]:
import requests


from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
import requests  # Adicionando a importação do módulo requests



# Faz o download da página
response = requests.get(url)
html = response.text
# URL da página com as tabelas
url = "ULR DA SUA PAGINA AQUI"


# Função para extrair os dados de uma tabela
def extract_table_data(table):
    data = []
    rows = table.find_all("tr")
    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.get_text(strip=True) for col in cols]
        data.append(cols)
    return data


# Função para extrair os links de uma tabela
def extract_table_links(table):
    links = []
    rows = table.find_all("tr")
    for row in rows:
        link = row.find("a")
        if link:
            links.append(link["href"])
        else:
            links.append("")
    return links


# Faz o download da página
response = requests.get(url)
html = response.text


# Analisa o HTML da página
soup = BeautifulSoup(html, "html.parser")


# Encontra todas as tabelas na página
tables = soup.find_all("table")


# Cria um arquivo XLSX e uma planilha
workbook = Workbook()
sheet = workbook.active


# Itera sobre as tabelas e seus dados
#Codigo feito por Cris Andrade
for table in tables:
    table_data = extract_table_data(table)
    table_links = extract_table_links(table)


    # Insere os dados na planilha
    for row_data, link in zip(table_data, table_links):
        if any(row_data):
            row_data.append(link)
            sheet.append(row_data)


# Adiciona cabeçalhos formatados
headers = []
for table in tables:
    headers.extend(table.find_all("th"))
    if headers:
        header_text = [header.get_text(strip=True) for header in headers]
        header_text.append("Links")
        sheet.append(header_text)
        break


# Formatação de cabeçalho em negrito
for row in sheet.iter_rows(min_row=1, max_row=1):
    for cell in row:
        cell.font = cell.font.copy(bold=True)


# Ajusta a largura das colunas
for column_cells in sheet.columns:
    max_length = 0
    for cell in column_cells:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    sheet.column_dimensions[column_cells[0].column_letter].width = adjusted_width


# Salva o arquivo XLSX
output_filename = "NOME DO SEU AQUIVO.xlsx"
workbook.save(output_filename)


print(f"Dados extraídos e salvos em '{output_filename}'")

